In [ ]:
from ortools.sat.python import cp_model

In [ ]:
import marimo as mo
import nbformat
import util

# MILP 定式化 (CP-SAT)

数理最適化モデルを用いて SCSP を解く.
定式化については SCIP 版を参照.

In [ ]:
class Model:
    def __init__(self, instance: list[str]):
        max_len = sum(len(s) for s in instance)

        cpmodel = cp_model.CpModel()

        seqs = [
            [
                cpmodel.new_int_var(0, max_len - 1, "")
                for c in s
            ]
            for s in instance
        ]

        for seq in seqs:
            for idx, _ in enumerate(seq):
                if idx == 0:
                    continue
                cpmodel.add(seq[idx - 1] < seq[idx])

        for idx1, (s1, seq1) in enumerate(zip(instance, seqs)):
            for idx2, (s2, seq2) in enumerate(zip(instance, seqs)):
                if idx1 >= idx2:
                    continue
                for cidx1, (c1, cvar1) in enumerate(zip(s1, seq1)):
                    for cidx2, (c2, cvar2) in enumerate(zip(s2, seq2)):
                        if c1 != c2:
                            cpmodel.add(cvar1 != cvar2)

        obj = cpmodel.new_int_var(0, max_len, "")
        cpmodel.add_max_equality(obj, [seq[-1] for seq in seqs])
        cpmodel.minimize(obj)

        self.instance = instance
        self.cpmodel = cpmodel
        self.cpsolver = cp_model.CpSolver()
        self.seqs = seqs
        self.status = None

    def solve(self, time_limit: int | None = 60) -> "Model":
        self.cpsolver.parameters.log_search_progress = True
        if time_limit is not None:
            self.cpsolver.parameters.max_time_in_seconds = time_limit
        self.status = self.cpsolver.solve(self.cpmodel)

        return self

    def to_solution(self) -> str | None:
        if self.status not in {cp_model.cp_model_pb2.OPTIMAL, cp_model.cp_model_pb2.FEASIBLE}:
            return None

        objval = int(self.cpsolver.objective_value)
        sol_char_idx = 0
        solution = ""
        while sol_char_idx <= objval:
            found = False
            for idx, (s, seq) in enumerate(zip(self.instance, self.seqs)):
                for c_idx, cvar in enumerate(seq):
                    if self.cpsolver.value(cvar) == sol_char_idx:
                        solution += s[c_idx]
                        found = True
                        sol_char_idx += 1
                if found:
                    break
            if not found:
                sol_char_idx += 1

        return solution

In [ ]:
def solve(instance: list[str], time_limit: int | None = 60) -> str | None:
    return Model(instance).solve(time_limit).to_solution()

In [ ]:
instance_01 = util.parse("uniform_q26n004k015-025.txt")
solution_01 = solve(instance_01)


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 60 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0xc79980cb1af75797)
#Variables: 85 (#ints: 1 in objective) (84 primary variables)
  - 84 in [0,83]
  - 1 in [0,84]
#kLinMax: 1 (#expressions: 4)
#kLinear2: 2'607 (#complex_domain: 2'527)

Starting presolve at 0.00s


  1.84e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.84e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=25 #num_dual_strengthening=1 
  2.03e-04s  0.00e+00d  [ExtractEncodingFromLinear] 
  1.85e-04s  0.00e+00d  [DetectDuplicateColumns] 
  4.32e-05s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 2'780 nodes and 5'306 arcs.
[Symmetry] Symmetry computation done. time: 0.000288547 dtime: 0.00039434


  5.18e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.74e-01s  5.43e-02d  [Probe] #probed=10'108 #new_binary_clauses=554 
  5.69e-06s  0.00e+00d  [MaxClique] 
  1.63e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.01e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  2.93e-04s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  4.32e-05s  0.00e+00d  [DetectDuplicateConstraints] 
  4.59e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  3.01e-04s  1.89e-05d  [DetectDominatedLinearConstraints] #relevant_constraints=2'611 


  1.07e-03s  0.00e+00d  [DetectDifferentVariables] #different=2'607 #cliques=71 #size=547 
  1.05e-05s  0.00e+00d  [ProcessSetPPC] 
  9.46e-06s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  2.65e-06s  0.00e+00d  [FindBigAtMostOneAndLinearOverlap] 
  3.13e-04s  1.33e-04d  [FindBigVerticalLinearOverlap] 
  7.63e-06s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  5.19e-06s  0.00e+00d  [MergeClauses] 
  1.75e-04s  0.00e+00d  [DetectDominanceRelations] 
  2.69e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  1.70e-04s  0.00e+00d  [DetectDominanceRelations] 
  2.67e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  5.62e-05s  0.00e+00d  [DetectDuplicateColumns] 
  7.75e-05s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 2'935 nodes and 5'937 arcs.
[Symmetry] Symmetry computation done. time: 0.000193938 dtime: 0.00043999


  1.09e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  2.35e-01s  5.79e-02d  [Probe] #probed=10'108 #new_binary_clauses=554 
  9.10e-06s  0.00e+00d  [MaxClique] 
  1.80e-04s  0.00e+00d  [DetectDominanceRelations] 
  2.77e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  2.92e-04s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  8.26e-05s  0.00e+00d  [DetectDuplicateConstraints] 
  7.71e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  3.05e-04s  1.89e-05d  [DetectDominatedLinearConstraints] #relevant_constraints=2'611 
  3.15e-04s  0.00e+00d  [DetectDifferentVariables] 
  9.39e-06s  0.00e+00d  [ProcessSetPPC] 
  9.18e-06s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  2.69e-06s  0.00e+00d  [FindBigAtMostOneAndLinearOverlap] 
  3.11e-04s  1.33e-04d  [FindBigVerticalLinearOverlap] 
  6.90e-06s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  5.40e-06s  0.00e+00d  [MergeClauses] 
  1.72e-04s  0.00e+00d  [Detec

...

#16      6.81s best:67    next:[29,66]    rins_pump_lns (d=5.29e-01 s=151 t=0.10 p=0.43 stall=0 h=base)


#17      7.86s best:65    next:[29,64]    graph_var_lns (d=5.29e-01 s=179 t=0.10 p=0.43 stall=0 h=base)


#Bound  23.39s best:65    next:[30,64]    reduced_costs


#18     55.06s best:64    next:[30,63]    graph_var_lns (d=7.11e-01 s=1211 t=0.10 p=0.51 stall=0 h=base)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
           'default_lp':         1 [  59.51s,   59.51s]   59.51s   0.00ns   59.51s         1 [  13.75s,   13.75s]   13.75s   0.00ns   13.75s
     'feasibility_pump':       270 [ 10.22us,   7.13ms]  99.73us 430.88us  26.93ms       267 [ 11.64us,  64.63us]  11.84us   3.24us   3.16ms
                   'fj':         4 [ 60.15ms,  68.36ms]  64.55ms   3.19ms 258.22ms         4 [100.13ms, 101.21ms] 100.44ms 444.66us 401.75ms
                   'fj':         4 [ 61.23ms,  67.21ms]  65.03ms   2.29ms 260.13ms         4 [100.13ms, 101.14ms] 100.62ms 381.18us 402.49ms
            'fs_random':         1 [238.35ms, 238.35ms] 238.35ms   0.00ns 238.35ms         1 [  1.41ms,   1.41ms]   1.41ms   0.00ns   1.41ms
      'fs_random_no_lp':         1 [238.12ms, 238.12ms] 238.12ms   0.00ns 238.12ms         1 [  1.33ms,   1.33ms]   1.33ms   0.00ns   1.33ms
        'gra

In [ ]:
_instance = instance_01
_solution = solution_01

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 65) ---
 Sol: tigevauzkgbrldcdbinycojsiqfolnvbrxxcvnksuhmpxnhtqogxzpvxissbpxflp
str1: t-------kg--------n-------------------k-uhmpxnhtq-gxz-vxis-------
str2: -i-------------------oj-iqfoln-b-xxcv--su-------q----pv-issb-xf--
str3: ------u-----l-c--inyco-s---o--v------------------o--zp------p--lp
str4: -igeva-z-gbr-d-db---c--s------v-r---vn-------n----g-----------f--

solution is feasible: True


In [ ]:
instance_02 = util.parse("uniform_q26n008k015-025.txt")
solution_02 = solve(instance_02)


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 60 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x3a94167cd68f9e7f)
#Variables: 176 (#ints: 1 in objective) (175 primary variables)
  - 175 in [0,174]
  - 1 in [0,175]
#kLinMax: 1 (#expressions: 8)
#kLinear2: 12'970 (#complex_domain: 12'803)

Starting presolve at 0.00s


  7.78e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.89e-01s  0.00e+00d  [PresolveToFixPoint] #num_loops=25 #num_dual_strengthening=1 
  4.55e-05s  0.00e+00d  [ExtractEncodingFromLinear] 
  8.70e-04s  0.00e+00d  [DetectDuplicateColumns] 
  2.32e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 13'329 nodes and 26'131 arcs.
[Symmetry] Symmetry computation done. time: 0.00136678 dtime: 0.0024951


  3.36e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  2.17e+00s  5.61e-01d  [Probe] #probed=51'212 #new_bounds=1 
  7.53e-05s  0.00e+00d  [MaxClique] 


  8.01e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.33e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  1.46e-03s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  2.75e-04s  0.00e+00d  [DetectDuplicateConstraints] 
  2.68e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  2.56e-03s  1.32e-04d  [DetectDominatedLinearConstraints] #relevant_constraints=12'978 
  3.63e-03s  0.00e+00d  [DetectDifferentVariables] #different=12'970 #cliques=148 #size=1'985 
  5.13e-05s  0.00e+00d  [ProcessSetPPC] 
  4.69e-05s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  5.28e-06s  0.00e+00d  [FindBigAtMostOneAndLinearOverlap] 
  1.62e-03s  6.59e-04d  [FindBigVerticalLinearOverlap] 
  3.32e-05s  0.00e+00d  [FindBigHorizontalLinearOverlap] 


  2.42e-05s  0.00e+00d  [MergeClauses] 


...

#19     35.60s best:138   next:[32,137]   graph_var_lns (d=3.91e-01 s=461 t=0.10 p=0.43 stall=0 h=base)


#20     45.98s best:135   next:[32,134]   graph_var_lns (d=3.74e-01 s=612 t=0.10 p=0.45 stall=0 h=base)


#21     52.15s best:134   next:[32,133]   graph_cst_lns (d=6.95e-01 s=664 t=0.10 p=0.55 stall=0 h=base)


#22     55.00s best:132   next:[32,131]   rnd_var_lns (d=9.46e-01 s=681 t=0.10 p=0.60 stall=0 h=base)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
           'default_lp':         1 [  53.65s,   53.65s]   53.65s   0.00ns   53.65s         1 [   7.00s,    7.00s]    7.00s   0.00ns    7.00s
     'feasibility_pump':       224 [  6.73us,  28.57ms] 245.27us   1.90ms  54.94ms       217 [ 24.15us, 161.44us]  24.78us   9.30us   5.38ms
                   'fj':        15 [ 63.95ms,  95.44ms]  77.26ms   8.64ms    1.16s        15 [100.06ms, 101.12ms] 100.61ms 355.29us    1.51s
                   'fj':        15 [ 66.32ms,  95.92ms]  79.53ms   8.44ms    1.19s        15 [100.11ms, 100.96ms] 100.62ms 276.26us    1.51s
            'fs_random':         1 [   2.43s,    2.43s]    2.43s   0.00ns    2.43s         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_random_no_lp':         1 [   2.43s,    2.43s]    2.43s   0.00ns    2.43s         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
        'gra

In [ ]:
_instance = instance_02
_solution = solution_02

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 133) ---
 Sol: rigevazxgbwxqkrdrlctodtmprpknbxwczfjtvxerzbrvigdevdcvdpfzsymsbrouplrzxucpmqvgtdfuivcdebcsivnycrolsvnboxkxcvsouzhpmplxnhtqpgxzvxissbxf
str1: -------------------t-------k------------------g--------------------------------------------n-----------k-----u-h-mp-xnhtq-gxzvxis----
str2: -i------------------o--------------j---------i----------------------------q----f---------------ol--nb-x-xcvs-u----------qp---v-issbxf
str3: ----------------------------------------------------------------u-l----c---------i---------nyc-o-s---o----v-o-z-p-pl-----p-----------
str4: -igevaz-gb----rd-----d-------b--c------------------------s-----------------v------------------r---vn-----------------n----g---------f
s

In [ ]:
instance_03 = util.parse("uniform_q26n016k015-025.txt")
solution_03 = solve(instance_03)


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 60 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0xee9bc41fa0f95ee6)
#Variables: 324 (#ints: 1 in objective) (323 primary variables)
  - 323 in [0,322]
  - 1 in [0,323]
#kLinMax: 1 (#expressions: 16)
#kLinear2: 47'165 (#complex_domain: 46'858)

Starting presolve at 0.01s


  2.75e-03s  0.00e+00d  [DetectDominanceRelations] 
  6.57e-01s  0.00e+00d  [PresolveToFixPoint] #num_loops=25 #num_dual_strengthening=1 
  2.28e-04s  0.00e+00d  [ExtractEncodingFromLinear] 


  3.30e-03s  0.00e+00d  [DetectDuplicateColumns] 
  1.08e-03s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 47'828 nodes and 94'685 arcs.


[Symmetry] Symmetry computation done. time: 0.00536159 dtime: 0.00957173


  1.20e-03s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  4.08e+00s  1.00e+00d *[Probe] #probed=51'616 #new_bounds=1 
  1.78e-04s  0.00e+00d  [MaxClique] 


...

#13     48.57s best:288   next:[26,287]   ls_lin_restart_decay_perturb(batch:1 lin{mvs:367 evals:1'530} #w_updates:109 #perturb:0)


#14     49.43s best:287   next:[26,286]   ls_lin_restart_perturb(batch:1 lin{mvs:516 evals:2'632} #w_updates:307 #perturb:0)


#15     53.44s best:286   next:[26,285]   ls_restart_decay(batch:1 lin{mvs:406 evals:1'843} #w_updates:110 #perturb:0)


#16     59.44s best:285   next:[26,284]   ls_lin_restart_perturb(batch:1 lin{mvs:175 evals:800} #w_updates:121 #perturb:0)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
           'default_lp':         1 [  41.91s,   41.91s]   41.91s   0.00ns   41.91s         1 [531.08ms, 531.08ms] 531.08ms   0.00ns 531.08ms
     'feasibility_pump':       117 [ 20.43us, 130.98ms]   1.20ms  12.05ms 140.41ms        17 [ 44.48us, 373.35us]  63.82us  77.38us   1.08ms
                   'fj':         9 [125.82ms, 241.70ms] 206.75ms  34.39ms    1.86s         9 [100.29ms, 102.49ms] 102.06ms 688.83us 918.57ms
                   'fj':         9 [148.66ms, 285.67ms] 200.72ms  44.06ms    1.81s         9 [100.31ms, 103.32ms] 102.15ms   1.02ms 919.36ms
            'fs_random':         1 [   7.00s,    7.00s]    7.00s   0.00ns    7.00s         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_random_no_lp':         1 [   6.99s,    6.99s]    6.99s   0.00ns    6.99s         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
        'gra

In [ ]:
_instance = instance_03
_solution = solution_03

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

-------------------k-------------r---------i----------------------f----------savncdqwh--z-------------------------------c-----------------------------------------------------------------------------------------------------------------------------------------------
str12: --------------------------q-------------a----------------x--------u------------dgqvqcewbfgijoww---------------------------------------------------------------------------------------------------y-------------------------------------------------------------------------------------------
str13: -------------------------------------------------------rsxqjnfpadiusiqbezhko-------------------------------h---------------------------------------m----------------------g-------------------------------------------------------------------------------------------------------------------
str14: ------i------------w-----------------------shvhcomiuvdd-------------------------------------------------------------------------------------

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 286) ---
  Sol: lxxpabivusfcfzpeecvwantfmgqzhtxxqjzqbctbakwshvhcomiuvddrsxqjnfpadiusiqbezhkoaxudgqvqcewbfgijowwkrifsavncdqwhbvzkozzkqafigqjwokkskrblrxwxqkrdrlctodtmprpxwenbczfjtvxerzbrvigplbdevdcvdpfzsmsbroqvbbyplrzxucpmqvgtdfuiovcdsgevazgbrddbcsvrvntlcinycosovozkgjppiqfolnkbxxcvsuhmpxnhtqgxzpvxissbxf
str01: ----------------------t------------------k--------------------------------------g---------------------n--------k--------------------------------------------------------------------

-----------------------------------------------------------------------------------------------------------
str15: ----------------------------htxxqjzqbctbak------------------n---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str16: -x------usfcfzpeecvwantfmgqz-----------------------u------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

solution is feasible: True


In [ ]:
instance_04 = util.parse("uniform_q05n010k010-010.txt")
solution_04 = solve(instance_04)


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 60 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0xe8b91a3b1b284b47)
#Variables: 101 (#ints: 1 in objective) (100 primary variables)
  - 100 in [0,99]
  - 1 in [0,100]
#kLinMax: 1 (#expressions: 10)
#kLinear2: 3'705 (#complex_domain: 3'615)

Starting presolve at 0.00s


  2.48e-04s  0.00e+00d  [DetectDominanceRelations] 
  2.55e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=10 #num_dual_strengthening=1 
  1.43e-05s  0.00e+00d  [ExtractEncodingFromLinear] 
  1.37e-04s  0.00e+00d  [DetectDuplicateColumns] 
  1.27e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 3'916 nodes and 7'530 arcs.
[Symmetry] Symmetry computation done. time: 0.000404408 dtime: 0.0009157


  1.74e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.60e-01s  4.68e-02d  [Probe] #probed=14'460 #new_bounds=1 
  7.09e-06s  0.00e+00d  [MaxClique] 
  2.27e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.74e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  4.12e-04s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  1.49e-04s  0.00e+00d  [DetectDuplicateConstraints] 
  1.57e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  4.90e-04s  2.84e-05d  [DetectDominatedLinearConstraints] #relevant_constraints=3'715 
  9.63e-04s  0.00e+00d  [DetectDifferentVariables] #different=3'705 #cliques=84 #size=506 
  1.46e-05s  0.00e+00d  [ProcessSetPPC] 
  1.58e-05s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  4.12e-06s  0.00e+00d  [FindBigAtMostOneAndLinearOverlap] 
  4.46e-04s  1.88e-04d  [FindBigVerticalLinearOverlap] 
  8.97e-06s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  1.14e-05s  0.00e+00d  [MergeClauses] 


  2.41e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.83e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  2.35e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.75e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  8.01e-05s  0.00e+00d  [DetectDuplicateColumns] 
  1.76e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 4'100 nodes and 8'136 arcs.
[Symmetry] Symmetry computation done. time: 0.00040553 dtime: 0.00097168


  2.04e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  2.03e-01s  5.00e-02d  [Probe] #probed=14'460 
  9.63e-06s  0.00e+00d  [MaxClique] 
  2.45e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.85e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  4.16e-04s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  1.64e-04s  0.00e+00d  [DetectDuplicateConstraints] 
  1.84e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  4.82e-04s  2.84e-05d  [DetectDominatedLinearConstraints] #relevant_constraints=3'715 
  4.58e-04s  0.00e+00d  [DetectDifferentVariables] 
  1.25e-05s  0.00e+00d  [ProcessSetPPC] 
  1.26e-05s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  3.05e-06s  0.00e+00d  [FindBigAtMostOneAndLinearOverlap] 
  4.34e-04s  1.88e-04d  [FindBigVerticalLinearOverlap] 
  9.06e-06s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  6.33e-06s  0.00e+00d  [MergeClauses] 
  2.38e-04s  0.00e+00d  [DetectDominanceRelations] 
  

...

#60      5.72s best:30    next:[15,29]    rnd_cst_lns (d=9.39e-01 s=113 t=0.10 p=1.00 stall=0 h=base)


#61      7.39s best:29    next:[15,28]    graph_var_lns (d=9.13e-01 s=148 t=0.10 p=0.83 stall=0 h=base)


#Bound   9.48s best:29    next:[16,28]    reduced_costs


#62     15.21s best:28    next:[16,27]    graph_var_lns (d=9.15e-01 s=291 t=0.10 p=0.70 stall=0 h=base)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
           'default_lp':         1 [  59.57s,   59.57s]   59.57s   0.00ns   59.57s         1 [   7.75s,    7.75s]    7.75s   0.00ns    7.75s
     'feasibility_pump':       266 [  3.78us,   8.69ms] 108.83us 528.52us  28.95ms       254 [ 13.78us,  73.96us]  14.01us   3.77us   3.56ms
                   'fj':         4 [ 62.17ms,  66.89ms]  65.14ms   1.80ms 260.56ms         4 [100.21ms, 100.92ms] 100.41ms 298.45us 401.63ms
                   'fj':         4 [ 62.48ms,  65.22ms]  64.35ms   1.09ms 257.39ms         4 [100.21ms, 100.46ms] 100.31ms 101.92us 401.25ms
            'fs_random':         1 [208.88ms, 208.88ms] 208.88ms   0.00ns 208.88ms         1 [930.89us, 930.89us] 930.89us   0.00ns 930.89us
      'fs_random_no_lp':         1 [211.31ms, 211.31ms] 211.31ms   0.00ns 211.31ms         1 [  1.84ms,   1.84ms]   1.84ms   0.00ns   1.84ms
        'gra

In [ ]:
_instance = instance_04
_solution = solution_04

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 29) ---
  Sol: bbaecdacedbeedccbeadbcecebdad
str01: -----d-c--b---cc---dbc-ce----
str02: b----d---dbee----e----e--bd--
str03: ----c-ac-d-ee-c--e--b-e------
str04: --ae-d---d---d-----d--e--bd-d
str05: --a-c-----bee-c---a-bce------
str06: bba-------be----b--d-c---b-a-
str07: bbae--a-e-b-------ad-------a-
str08: ---e----e--ee-c-b--db-e-e----
str09: ----c--c-d-eed----ad-c----d--
str10: b----da---b--d--bea--------ad

solution is feasible: True


In [ ]:
instance_05 = util.parse("uniform_q05n050k010-010.txt")
solution_05 = solve(instance_05)


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 60 log_search_progress: true
Setting number of workers to 12



Initial optimization model '': (model_fingerprint: 0xfb1fbec87aa5a80b)
#Variables: 501 (#ints: 1 in objective) (500 primary variables)
  - 500 in [0,499]
  - 1 in [0,500]
#kLinMax: 1 (#expressions: 50)
#kLinear2: 98'481 (#complex_domain: 98'031)

Starting presolve at 0.02s


  5.99e-03s  0.00e+00d  [DetectDominanceRelations] 
  6.98e-01s  0.00e+00d  [PresolveToFixPoint] #num_loops=10 #num_dual_strengthening=1 
  8.13e-04s  0.00e+00d  [ExtractEncodingFromLinear] 


  5.66e-03s  0.00e+00d  [DetectDuplicateColumns] 
  2.89e-03s  0.00e+00d  [DetectDuplicateConstraints] 


[Symmetry] Graph for symmetry has 99'532 nodes and 197'562 arcs.


...

#25     56.11s best:468   next:[15,467]   default_lp


#26     57.02s best:467   next:[15,466]   default_lp


#27     57.93s best:466   next:[15,465]   default_lp


#28     58.85s best:465   next:[15,464]   default_lp



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
           'default_lp':         1 [  48.58s,   48.58s]   48.58s   0.00ns   48.58s         1 [   2.83s,    2.83s]    2.83s   0.00ns    2.83s
     'feasibility_pump':        42 [202.92us, 288.69ms]   7.21ms  43.96ms 302.80ms        41 [ 68.50us, 670.84us]  83.19us  92.92us   3.41ms
                   'fj':        21 [105.30ms, 428.11ms] 204.95ms  78.04ms    4.30s        21 [101.23ms, 105.64ms] 104.36ms   1.63ms    2.19s
                   'fj':        23 [118.39ms, 467.54ms] 201.37ms  77.19ms    4.63s        23 [100.98ms, 105.64ms] 104.17ms   1.72ms    2.40s
            'fs_random':         1 [  12.01s,   12.01s]   12.01s   0.00ns   12.01s         1 [472.82ms, 472.82ms] 472.82ms   0.00ns 472.82ms
      'fs_random_no_lp':         1 [  12.05s,   12.05s]   12.05s   0.00ns   12.05s         1 [277.69ms, 277.69ms] 277.69ms   0.00ns 277.69ms
        'gra

In [ ]:
_instance = instance_05
_solution = solution_05

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution (of length 466) ---
  Sol: adacedaedabdabcecbbaeeebbd

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str08: ----e--e-------e----e------c--b----d----b-e-------e-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str09: ---c----------c----------d---e---e-da--d-----c-d-----------------------------------------------------------------------------------------------------

------------------------------------------------------------------------------------------------------------------------------
str14: ----------bd---e----e-------a------d-ead--e---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str15: ---c--aeda-d---e----ee---d------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str21: ----------b--b-----------d--ae-c----a-ad--e---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str22: -dacedaedab----------------------------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------
str27: ---ce--e--b---c----------d-c--b----d-e--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str28: -d--------b----e---------d--a-------a--d----a----a----b---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str34: a-a-e-a---b-a------ae--b----a-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str35: ----e---------c--bb--------ca-------a--d-----c-d----------------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------------------
str40: -dac------b--b-----------d-c-e-----d---------c------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str41: -d--ed----b----e----e--bbd---e------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str47: -d--------bd---e-b-a-------c---c---d----b-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
str48: ----e-----b---c--b--ee---d--ae------a--------------------------------------------------------------------------------------------------------------------

In [ ]:
instance_06 = util.parse("nucleotide_n010k010.txt")
solution_06 = solve(instance_06)


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 60 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x13ba79668ddf2740)
#Variables: 101 (#ints: 1 in objective) (100 primary variables)
  - 100 in [0,99]
  - 1 in [0,100]
#kLinMax: 1 (#expressions: 10)
#kLinear2: 3'429 (#complex_domain: 3'339)

Starting presolve at 0.00s


  2.32e-04s  0.00e+00d  [DetectDominanceRelations] 
  2.33e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=10 #num_dual_strengthening=1 
  1.33e-05s  0.00e+00d  [ExtractEncodingFromLinear] 
  9.81e-05s  0.00e+00d  [DetectDuplicateColumns] 
  5.96e-05s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 3'640 nodes and 6'978 arcs.
[Symmetry] Symmetry computation done. time: 0.000355304 dtime: 0.00085275


  9.68e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.42e-01s  4.09e-02d  [Probe] #probed=13'356 #new_bounds=1 
  1.05e-05s  0.00e+00d  [MaxClique] 
  2.22e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.52e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 


  3.81e-04s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  6.28e-05s  0.00e+00d  [DetectDuplicateConstraints] 
  6.33e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  4.34e-04s  2.58e-05d  [DetectDominatedLinearConstraints] #relevant_constraints=3'439 
  8.54e-04s  0.00e+00d  [DetectDifferentVariables] #different=3'429 #cliques=85 #size=411 
  1.22e-05s  0.00e+00d  [ProcessSetPPC] 
  1.15e-05s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  3.10e-06s  0.00e+00d  [FindBigAtMostOneAndLinearOverlap] 
  4.05e-04s  1.74e-04d  [FindBigVerticalLinearOverlap] 
  8.42e-06s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  7.47e-06s  0.00e+00d  [MergeClauses] 
  2.17e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.54e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  2.18e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.46e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  7.34e-05s  0.00e+00d  [DetectDuplicateCo

  1.06e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.61e-01s  4.33e-02d  [Probe] #probed=13'356 
  7.68e-06s  0.00e+00d  [MaxClique] 
  2.23e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.48e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  3.82e-04s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  8.64e-05s  0.00e+00d  [DetectDuplicateConstraints] 
  9.04e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  4.28e-04s  2.58e-05d  [DetectDominatedLinearConstraints] #relevant_constraints=3'439 
  4.08e-04s  0.00e+00d  [DetectDifferentVariables] 
  1.10e-05s  0.00e+00d  [ProcessSetPPC] 
  1.14e-05s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  2.96e-06s  0.00e+00d  [FindBigAtMostOneAndLinearOverlap] 
  4.02e-04s  1.74e-04d  [FindBigVerticalLinearOverlap] 
  9.10e-06s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  5.96e-06s  0.00e+00d  [MergeClauses] 
  2.15e-04s  0.00e+00d  [DetectDominanceRelations] 
  

...

#Model  49.45s var:3428/3440 constraints:6851/6863


#Model  54.39s var:3427/3440 constraints:6850/6863


#Model  57.66s var:3426/3440 constraints:6849/6863


#Model  59.77s var:3425/3440 constraints:6848/6863



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
           'default_lp':         1 [  59.63s,   59.63s]   59.63s   0.00ns   59.63s         1 [  10.18s,   10.18s]   10.18s   0.00ns   10.18s
     'feasibility_pump':       268 [  4.07us,   6.36ms] 101.27us 385.07us  27.14ms       258 [ 13.78us,  73.96us]  14.01us   3.74us   3.61ms
                   'fj':         3 [ 62.49ms,  94.96ms]  74.15ms  14.75ms 222.45ms         3 [100.20ms, 100.52ms] 100.36ms 128.36us 301.09ms
                   'fj':         3 [ 63.82ms,  71.17ms]  66.30ms   3.44ms 198.91ms         3 [100.19ms, 100.34ms] 100.26ms  63.28us 300.79ms
            'fs_random':         1 [170.62ms, 170.62ms] 170.62ms   0.00ns 170.62ms         1 [  1.67ms,   1.67ms]   1.67ms   0.00ns   1.67ms
      'fs_random_no_lp':         1 [170.27ms, 170.27ms] 170.27ms   0.00ns 170.27ms         1 [  1.69ms,   1.69ms]   1.69ms   0.00ns   1.69ms
        'gra

In [ ]:
_instance = instance_06
_solution = solution_06

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 4 chars) ---
str01: ATGGGATACG
str02: ATACCTTCCC
str03: CACGAATTGA
str04: TAAAATCTGT
str05: AGGTAACAAA
str06: TTCCTAGGTA
str07: TTGTAGATCT
str08: TGGGAAGTTC
str09: TTCCACAACT
str10: TCTAAACGAA

--- Solution (of length 24) ---
  Sol: ATGGTGACCTACGAATCTCCGTAA
str01: ATGG-GA--TACG-----------
str02: AT----ACCT-----TC-CC----
str03: -------C--ACGAAT-T--G-A-
str04: -T----A---A--AATCT--GT--
str05: A-GGT-A---AC-AA-------A-
str06: -T--T--CCTA-G-------GTA-
str07: -T--TG---TA-GA-TCT------
str08: -TGG-GA---A-G--T-TC-----
str09: -T--T--CC-AC-AA-CT------
str10: -T-----C-TA--AA-C---G-AA

solution is feasible: True


In [ ]:
instance_07 = util.parse("nucleotide_n050k050.txt")
solution_07 = solve(instance_07)


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 60 log_search_progress: true
Setting number of workers to 12


Initial optimization model '': (model_fingerprint: 0xd6817d8ed3086802)
#Variables: 2'501 (#ints: 1 in objective) (2'500 primary variables)
  - 2'500 in [0,2499]
  - 1 in [0,2500]
#kLinMax: 1 (#expressions: 50)
#kLinear2: 2'286'040 (#complex_domain: 2'283'590)

Starting presolve at 0.45s


  5.83e+01s  0.00e+00d  [PresolveToFixPoint] #num_loops=27 #num_dual_strengthening=1 



Presolve summary:
  - 0 affine relations were detected.
  - rule 'TODO dual: only one unspecified blocking constraint?' was applied 1 time.
  - rule 'lin_max: rewrite with precedences' was applied 1 time.
  - rule 'lin_max: target domain reduced' was applied 1 time.
  - rule 'linear: expanded complex rhs' was applied 2'283'590 times.
  - rule 'linear: reduced variable domains' was applied 48'600 times.
  - rule 'new_bool: complex linear expansion' was applied 2'283'590 times.
  - rule 'presolve: 0 unused variables removed.' was applied 1 time.
  - rule 'presolve: iteration' was applied 1 time.


Presolved optimization model '': (model_fingerprint: 0xfc857e20d7384fd3)
#Variables: 2'286'091 (#ints: 1 in objective) (2'286'090 primary variables)
  - 2'283'590 Booleans in [0,1]
  - 50 in [0,2472]
  - 50 in [1,2472]
  - 50 in [2,2472]
  - 50 in [3,2472]
  - 50 in [4,2472]
  - 50 in [5,2472]
  - 50 in [6,2472]
  - 50 in [7,2472]
  - 50 in [8,2472]
  - 50 in [9,2472]
  - 50 in [10,2472]
  - 50 in [11,2472]
  - 50 in [12,2472]
  - 50 in [13,2472]
  - 50 in [14,2472]
  - 50 in [15,2472]
  - 50 in [16,2472]
  - 50 in [17,2472]
  - 50 in [18,2472]
  - 50 in [19,2472]
  - 50 in [20,2472]
  - 50 in [21,2472]
  - 50 in [22,2472]
  - 50 in [23,2473]
  - 50 in [24,2474]
  - 50 in [25,2475]
  - 50 in [26,2476]
  - 50 in [27,2477]
  - 50 in [28,2478]
  - 50 in [29,2479]
  - 50 in [30,2480]
  - 50 in [31,2481]
  - 50 in [32,2482]
  - 50 in [33,2483]
  - 50 in [34,2484]
  - 50 in [35,2485]
  - 50 in [36,2486]
  - 50 in [37,2487]
  - 50 in [38,2488]
  - 50 in [39,2489]
  - 50 in [40,2490]
  - 50 in

Stopped after presolve.
PresolvedNumVariables: 2286091
PresolvedNumConstraints: 4569680
PresolvedNumTerms: 13706540
CpSolverResponse summary:
status: UNKNOWN
objective: 0
best_bound: 0
integers: 0
booleans: 0
conflicts: 0
branches: 0
propagations: 0
integer_propagations: 0
restarts: 0
lp_iterations: 0
walltime: 66.8381
usertime: 66.8381
deterministic_time: 0
gap_integral: 0



In [ ]:
_instance = instance_07
_solution = solution_07

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: TAGTAGTAGACTCCGGAAGTGACAAACCCTGAAAAGAATGGATAAATATA
str02: GGATAAACACTCCCGAAAATAATTTGACTTAAACAACGCGACAGTTCAAG
str03: ATACCTTCCTAGGTAACAAACCAACCAACTTTTGATCTCTTGTAGATCTG
str04: TAAATTATAATCTTATACTAGTAAAAAATAGGGTGTAACCGAAAACGGTC
str05: TTAAAACAGCCTGTGGGTTGCACCCACTCACAGGGCCCACTGGGCGCAAG
str06: ATGACTTCCAATGGATCCCAACCTCAAGCTTCCACCCCAATGGTTTCAGC
str07: AACAAACCAACCAACTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAAC
str08: ATGAAAACGAAAATTATTATCAAGGGTATGGAAGTGGAAGCTGACGAAAT
str09: ACTCGGCTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTA
str10: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str11: GCAGAGCATTTTCTAATATCCACAAAATGAAGGCAATAATTGTACTACTC
str12: ATGAGCCAAGATCCGACGAAGAGCCCCAAGGAGGAGAAGGAGGGACCCCC
str13: TCTCACAGTTCAAGAACCCAAAGTACCCCCCATAGCCCTCTTAAAGCCAC
str14: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str15: AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str16: TAAAACAACTCAATACAACATAAGAAAATCAACGCAAAAACACTCACAAA
str17: CCGCCCATTTGGGCGGCTCTCGAGCGATAGCT

In [ ]:
instance_08 = util.parse("protein_n010k010.txt")
solution_08 = solve(instance_08)


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 60 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x5928eb50e609d76f)
#Variables: 101 (#ints: 1 in objective) (100 primary variables)
  - 100 in [0,99]
  - 1 in [0,100]
#kLinMax: 1 (#expressions: 10)
#kLinear2: 4'341 (#complex_domain: 4'251)

Starting presolve at 0.00s


  2.66e-04s  0.00e+00d  [DetectDominanceRelations] 
  2.97e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=10 #num_dual_strengthening=1 
  1.63e-05s  0.00e+00d  [ExtractEncodingFromLinear] 
  2.76e-04s  0.00e+00d  [DetectDuplicateColumns] 
  7.12e-05s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 4'552 nodes and 8'802 arcs.
[Symmetry] Symmetry computation done. time: 0.000545856 dtime: 0.00106314


  9.68e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  2.11e-01s  6.15e-02d  [Probe] #probed=17'004 #new_bounds=1 
  1.19e-05s  0.00e+00d  [MaxClique] 
  2.68e-04s  0.00e+00d  [DetectDominanceRelations] 
  4.39e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  4.83e-04s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  7.75e-05s  0.00e+00d  [DetectDuplicateConstraints] 
  7.73e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  6.08e-04s  3.46e-05d  [DetectDominatedLinearConstraints] #relevant_constraints=4'351 
  1.36e-03s  0.00e+00d  [DetectDifferentVariables] #different=4'341 #cliques=69 #size=1'102 
  1.44e-05s  0.00e+00d  [ProcessSetPPC] 
  1.43e-05s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  3.05e-06s  0.00e+00d  [FindBigAtMostOneAndLinearOverlap] 
  5.15e-04s  2.23e-04d  [FindBigVerticalLinearOverlap] 
  1.01e-05s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  7.65e-06s  0.00e+00d  [MergeClauses] 

  2.84e-04s  0.00e+00d  [DetectDominanceRelations] 
  4.48e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  2.79e-04s  0.00e+00d  [DetectDominanceRelations] 
  4.45e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  9.34e-05s  0.00e+00d  [DetectDuplicateColumns] 
  1.33e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 4'721 nodes and 10'004 arcs.
[Symmetry] Symmetry computation done. time: 0.000437631 dtime: 0.00113292


  1.62e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  5.08e-01s  6.78e-02d  [Probe] #probed=17'004 
  1.03e-05s  0.00e+00d  [MaxClique] 
  2.83e-04s  0.00e+00d  [DetectDominanceRelations] 
  4.54e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  4.87e-04s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  1.33e-04s  0.00e+00d  [DetectDuplicateConstraints] 
  1.34e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  6.15e-04s  3.46e-05d  [DetectDominatedLinearConstraints] #relevant_constraints=4'351 
  5.28e-04s  0.00e+00d  [DetectDifferentVariables] 
  1.40e-05s  0.00e+00d  [ProcessSetPPC] 
  1.54e-05s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  3.10e-06s  0.00e+00d  [FindBigAtMostOneAndLinearOverlap] 
  5.15e-04s  2.23e-04d  [FindBigVerticalLinearOverlap] 
  1.03e-05s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  7.88e-06s  0.00e+00d  [MergeClauses] 


...

#35     16.10s best:53    next:[24,52]    graph_var_lns (d=6.17e-01 s=301 t=0.10 p=0.55 stall=0 h=base)


#36     21.68s best:52    next:[24,51]    graph_var_lns (d=5.21e-01 s=389 t=0.10 p=0.50 stall=0 h=base)


#37     22.61s best:51    next:[24,50]    graph_var_lns (d=6.92e-01 s=406 t=0.10 p=0.56 stall=0 h=base)


#38     24.88s best:49    next:[24,48]    rins_pump_lns (d=3.64e-01 s=471 t=0.10 p=0.40 stall=0 h=base)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
           'default_lp':         1 [  59.20s,   59.20s]   59.20s   0.00ns   59.20s         1 [   8.06s,    8.06s]    8.06s   0.00ns    8.06s
     'feasibility_pump':       253 [ 42.06us,  20.12ms] 157.15us   1.26ms  39.76ms       252 [ 13.78us,  73.96us]  14.01us   3.78us   3.53ms
                   'fj':         6 [ 64.33ms, 110.30ms]  79.87ms  17.08ms 479.20ms         6 [100.13ms, 100.43ms] 100.27ms  89.81us 601.64ms
                   'fj':         6 [ 65.93ms, 101.29ms]  78.30ms  13.01ms 469.78ms         6 [100.24ms, 100.42ms] 100.30ms  64.65us 601.82ms
            'fs_random':         1 [482.81ms, 482.81ms] 482.81ms   0.00ns 482.81ms         1 [  2.16ms,   2.16ms]   2.16ms   0.00ns   2.16ms
      'fs_random_no_lp':         1 [482.47ms, 482.47ms] 482.47ms   0.00ns 482.47ms         1 [  3.12ms,   3.12ms]   3.12ms   0.00ns   3.12ms
        'gra

In [ ]:
_instance = instance_08
_solution = solution_08

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 19 chars) ---
str01: MALSYCPKGT
str02: MQSSLNAIPV
str03: MPLSYQHFRK
str04: MEEHVNELHD
str05: MSNFDAIRAL
str06: MFRNQNSRNG
str07: MFYAHAFGGY
str08: MSKFTRRPYQ
str09: MSFVAGVTAQ
str10: MESLVPGFNE

--- Solution (of length 50) ---
  Sol: MESLKNFVPYAGEVFTHVDRANQNRIPELSYQHDSFCRGLNAIPKGTLYV
str01: M---------A-----------------LSY-----C------PKGT---
str02: M---------------------Q------S----S----LNAIP-----V
str03: M-------P-------------------LSYQH--F-R------K-----
str04: ME----------E---HV---N-----EL---HD----------------
str05: M-S--NF-----------D-A----I-----------R---A-----L--
str06: M-----F------------R-NQN-----S-------R--N----G----
str07: M-----F--YA-----H---A--------------F--G------G--Y-
str08: M-S-K-F--------T---R----R-P---YQ------------------
str09: M-S---FV--AG-V-T----A-Q---------------------------
str10: MESL---VP--G--F------N-----E----------------------

solution is feasible: True


In [ ]:
instance_09 = util.parse("protein_n050k050.txt")
solution_09 = solve(instance_09)    


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 60 log_search_progress: true
Setting number of workers to 12


Initial optimization model '': (model_fingerprint: 0x51ef83a55102100c)
#Variables: 2'501 (#ints: 1 in objective) (2'500 primary variables)
  - 2'500 in [0,2499]
  - 1 in [0,2500]
#kLinMax: 1 (#expressions: 50)
#kLinear2: 2'882'859 (#complex_domain: 2'880'409)

Starting presolve at 0.55s


  5.79e+01s  0.00e+00d  [PresolveToFixPoint] #num_loops=21 #num_dual_strengthening=1 



Presolve summary:
  - 0 affine relations were detected.
  - rule 'TODO dual: only one unspecified blocking constraint?' was applied 1 time.
  - rule 'lin_max: rewrite with precedences' was applied 1 time.
  - rule 'lin_max: target domain reduced' was applied 1 time.
  - rule 'linear: expanded complex rhs' was applied 2'880'409 times.
  - rule 'linear: reduced variable domains' was applied 40'950 times.
  - rule 'new_bool: complex linear expansion' was applied 2'880'409 times.
  - rule 'presolve: 0 unused variables removed.' was applied 1 time.
  - rule 'presolve: iteration' was applied 1 time.


Presolved optimization model '': (model_fingerprint: 0xa341dc7163e5bbfc)
#Variables: 2'882'910 (#ints: 1 in objective) (2'882'909 primary variables)
  - 2'880'409 Booleans in [0,1]
  - 50 in [0,2478]
  - 50 in [1,2478]
  - 50 in [2,2478]
  - 50 in [3,2478]
  - 50 in [4,2478]
  - 50 in [5,2478]
  - 50 in [6,2478]
  - 50 in [7,2478]
  - 50 in [8,2478]
  - 50 in [9,2478]
  - 50 in [10,2478]
  - 50 in [11,2478]
  - 50 in [12,2478]
  - 50 in [13,2478]
  - 50 in [14,2478]
  - 50 in [15,2478]
  - 50 in [16,2478]
  - 50 in [17,2478]
  - 50 in [18,2478]
  - 50 in [19,2478]
  - 50 in [20,2478]
  - 50 in [21,2478]
  - 50 in [22,2478]
  - 50 in [23,2478]
  - 50 in [24,2478]
  - 50 in [25,2478]
  - 50 in [26,2478]
  - 50 in [27,2478]
  - 50 in [28,2478]
  - 50 in [29,2479]
  - 50 in [30,2480]
  - 50 in [31,2481]
  - 50 in [32,2482]
  - 50 in [33,2483]
  - 50 in [34,2484]
  - 50 in [35,2485]
  - 50 in [36,2486]
  - 50 in [37,2487]
  - 50 in [38,2488]
  - 50 in [39,2489]
  - 50 in [40,2490]
  - 50 in

Stopped after presolve.
PresolvedNumVariables: 2882910
PresolvedNumConstraints: 5763318
PresolvedNumTerms: 17287454
CpSolverResponse summary:
status: UNKNOWN
objective: 0
best_bound: 0
integers: 0
booleans: 0
conflicts: 0
branches: 0
propagations: 0
integer_propagations: 0
restarts: 0
lp_iterations: 0
walltime: 68.5468
usertime: 68.5468
deterministic_time: 0
gap_integral: 0



In [ ]:
_instance = instance_09
_solution = solution_09

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 20 chars) ---
str01: MRHLNIDIETYSSNDIKNGVYKYADAEDFEILLFAYSIDGGEVECLDLTR
str02: MERRAHRTHQNWDATKPRERRKQTQHRLTHPDDSIYPRIEKAEGRKEDHG
str03: MEPGAFSTALFDALCDDILHRRLESQLRFGGVQIPPEVSDPRVYAGYALL
str04: MGKFYYSNRRLAVFAQAQSRHLGGSYEQWLACVSGDSAFRAEVKARVQKD
str05: FFRENLAFQQGKAREFPSEEARANSPTSRELWVRRGGNPLSEAGAERRGT
str06: MDPSLTQVWAVEGSVLSAAVDTAETNDTEPDEGLSAENEGETRIIRITGS
str07: MAFDFSVTGNTKLDTSGFTQGVSSMTVAAGTLIADLVKTASSQLTNLAQS
str08: MAVILPSTYTDGTAACTNGSPDVVGTGTMWVNTILPGDFFWTPSGESVRV
str09: MNTGIIDLFDNHVDSIPTILPHQLATLDYLVRTIIDENRSVLLFHIMGSG
str10: MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str11: MDSKETILIEIIPKIKSYLLDTNISPKSYNDFISRNKNIFVINLYNVSTI
str12: MLLSGKKKMLLDNYETAAARGRGGDERRRGWAFDRPAIVTKRDKSDRMAH
str13: MNGEEDDNEQAAAEQQTKKAKREKPKQARKVTSEAWEHFDATDDGAECKH
str14: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str15: MRYIVSPQLVLQVGKGQEVERALYLTPYDYIDEKSPIYYFLRSHLNIQRP
str16: MPRVPVYDSPQVSPNTVPQARLATPSFATPTFRGADAPAFQDTANQQARQ
str17: MFVFLVLLPLVSSQCVNLRTRTQLPLAYTNS